In [53]:
#Add langs to string based on picked data
langs = [x.lower() for x in ['Fr']]
#Generate predictions for Dev?
doDev = False
#packages
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, oob_score=True)

In [54]:
#read pickled data
with open('../Data/DuoData_processed.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [55]:
#build model function
def createModel(td):
    td = td.drop(['Unnamed: 0','WordFreq','?','?Pho','?EngPho'], axis = 1) #Always drop these unwanted columns
    continuousCols = list(td.select_dtypes(include=[np.number]))
    X_con = td[continuousCols] # Remove categorical values for now
    td = pd.concat([X_con,td[['source','instanceID']]], axis = 1,join_axes=[X_con.index])
    Xtrain = td[td['source'] != 'test'].drop(['source','instanceID'], axis = 1) # remove test data

    
    ytrain = Xtrain['error'] 
    Xtrain = Xtrain.drop(['error'], axis = 1)
    Xtrain = Xtrain.fillna(0)
    print('Training logistic regression...')
    rf.fit(Xtrain, ytrain)
     
    print('Generating predictions...')
    Xtest = td[td['source'] == 'test'].drop(['source'], axis = 1)
    Xtest = Xtest.fillna(0)
    Xtest = Xtest.drop(['error'],axis=1)
    Xtest['preds'] = [x[1] for x in rf.predict_proba(Xtest.drop(['instanceID'],axis = 1))]
    print('Done generating predictions!')
    return Xtest[['instanceID','preds']]

In [56]:
#build Dev model function
def createDevModel(td):

    td = td.drop(['Unnamed: 0','WordFreq','?','?Pho','?EngPho'], axis = 1) #Always drop these unwanted rows
    continuousCols = list(td.select_dtypes(include=[np.number]))
    X_con = td[continuousCols] # Remove categorical values for now
    td = pd.concat([X_con,td[['source','instanceID']]], axis = 1,join_axes=[X_con.index])
    Xtrain = td[td['source'] == 'train'].drop(['source','instanceID'], axis = 1) # remove test data

    
    ytrain = Xtrain['error'] 
    Xtrain = Xtrain.drop(['error'], axis = 1)
    Xtrain = Xtrain.fillna(0)
    print('Training logistic regression...')
    rf.fit(Xtrain, ytrain)
    
    print('Generating predictions...')
    Xtest = td[td['source'] == 'dev'].drop(['source'], axis = 1)
    Xtest = Xtest.fillna(0)
    Xtest = Xtest.drop(['error'],axis=1)
    Xtest['preds'] = [x[1] for x in rf.predict_proba(Xtest.drop(['instanceID'],axis = 1))]
    print('Done generating predictions!')
    return Xtest[['instanceID','preds']]

In [57]:
if doDev == False:
    for l2 in langs:
        data[l2+'_preds'] = createModel(data[l2+'_df'])

Training logistic regression...
Generating predictions...
Done generating predictions!


In [58]:
if doDev == True:
    for l2 in langs:
        data[l2+'_preds'] = createDevModel(data[l2+'_df'])

In [59]:
if doDev == False: 
    data[l2+'_preds'].to_csv('../Data/'+l2+'.txt', header=None, index=None, sep=' ')
elif doDev == True:
    data[l2+'_preds'].to_csv('../Data/'+l2+'_dev.txt', header=None, index=None, sep=' ')